In [1]:
import requests
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
token="vk1.a.nwCIBPvPnW-xmALXB0FS3Je2U9dTnNuuu-j1VyVsOFQiGK6PTFEYsGDOXG4pUY7UNafZN_TKaEEScuyocS9eylspJA6fwtf1vkJHc7HOuWNOEeEe51DxZthSROBKwxgMvx-GnYiwX2U8dS8ypHKgW6PK-_93w9-lYRqf8FFTM9A67_SZ5ZwTpNKk7NExamdZNhZ0tdE6NP1T424KxWT_gw"

In [3]:
group_name="kommersant"
offset=0
count=100
all_posts=[]
all_comments=[]

In [4]:
offset=0
while offset<1700:
        req=requests.get('https://api.vk.com/method/wall.search',
               params={
                   'access_token': token,
                   'v': 5.131,
                   'domain': group_name,
                   'owners_only': 1,
                   'query': 'инфляци',
                   'count': count,
                   'offset': offset
                   
               })
        res=req.json()['response']['items']
        offset+=100
        time.sleep(1)
        all_posts.extend(res)

In [5]:
res=req.json()['response']['items']
all_posts.extend(res)
df = pd.json_normalize(all_posts)

In [6]:
df.to_csv('Неочищенные посты Коммерсант.csv', index=False)

In [7]:
df=df[["id", "owner_id", "date", "text", "comments.count"]]
df.rename(columns={'date': 'post_time','text':'post_text'}, inplace=True)
df=df[(df['post_time']>=1630454400) & (df['post_time']<1675209600)]
df

,id,owner_id,post_time,post_text,comments.count
1,2856746,-23482909,1675193400,Рекордный спрос на золото на мировом рынке в 2...,4
2,2853257,-23482909,1674977912,,5
3,2852574,-23482909,1674922561,Исследователи маркетингово-коммуникационной ко...,2
4,2851076,-23482909,1674836092,Президент России Владимир Путин поручил поднят...,216
5,2850080,-23482909,1674766800,По итогам 2022 года номинальный рост рынка FMC...,0
...,...,...,...,...,...
532,2051577,-23482909,1631112014,"США, имеющие два года подряд дефицит бюджета 1...",18
533,2049650,-23482909,1630891270,Продолжение повышенной мировой инфляции после ...,1
534,2049603,-23482909,1630885271,Электросетевой госхолдинг «Россети» после мног...,2
535,2047811,-23482909,1630570231,Президент России Владимир Путин регулярно обсу...,19


In [8]:
owner_id=-23482909
posts_id=list(df[df['comments.count']!=0]['id'])

In [9]:
for post_id in posts_id:
    req=requests.get('https://api.vk.com/method/wall.getComments',
               params={
                   'access_token': token,
                   'v': 5.131,
                   'owner_id': owner_id,
                   'post_id': post_id,
                   'count': 100
               })
    time.sleep(1)
    res=req.json()['response']['items']
    all_comments.extend(res)

In [10]:
data = pd.json_normalize(all_comments)
data

,id,from_id,date,text,post_id,owner_id,parents_stack,thread.count,thread.items,thread.can_post,thread.show_reply_button,deleted,attachments
0,2856751,773582573,1675193535,"не только уйти от доллара, но и от рубля",2856746.0,-23482909.0,[],0,[],True,True,NaN,NaN
1,2856753,352750122,1675193549,Простым россиянам на золото разве что в ломбар...,2856746.0,-23482909.0,[],0,[],True,True,NaN,NaN
2,2857005,765652761,1675198624,Так скоро и до бартера дойдём,2856746.0,-23482909.0,[],0,[],True,True,NaN,NaN
3,2857123,262913240,1675201543,Сколько добывают и добыли за всю известную ист...,2856746.0,-23482909.0,[],0,[],True,True,NaN,NaN
4,2853263,540998671,1674978313,Тоже мне ...))\nРаньше выживали 500руб в недел...,2853257.0,-23482909.0,[],0,[],True,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3472,2047961,302868718,1630588204,,2047811.0,-23482909.0,[],0,[],True,True,NaN,"[{'type': 'video', 'video': {'access_key': '28..."
3473,2048037,201281732,1630597064,"Ну вот видите! Если бы не Путин, то цены вырос...",2047811.0,-23482909.0,[],0,[],True,True,NaN,NaN
3474,2048078,644528688,1630601474,Вообще-то это прерогатива правительства и неко...,2047811.0,-23482909.0,[],0,[],True,True,NaN,NaN
3475,2048130,577436947,1630607164,"Путин наш отец родной, об народе думает днём и...",2047811.0,-23482909.0,[],0,[],True,True,NaN,NaN


In [11]:
data.to_csv('Неочищенные комментарии Коммерсант.csv', index=False)

In [12]:
data=data[["post_id", "text", "date", "from_id"]]
data.rename(columns={'date': 'comment_time', 'id': 'comment_id'}, inplace=True)
data=data.dropna()
data['post_id'] = data['post_id'].astype(int)
data

,post_id,text,comment_time,from_id
0,2856746,"не только уйти от доллара, но и от рубля",1675193535,773582573
1,2856746,Простым россиянам на золото разве что в ломбар...,1675193549,352750122
2,2856746,Так скоро и до бартера дойдём,1675198624,765652761
3,2856746,Сколько добывают и добыли за всю известную ист...,1675201543,262913240
4,2853257,Тоже мне ...))\nРаньше выживали 500руб в недел...,1674978313,540998671
...,...,...,...,...
3472,2047811,,1630588204,302868718
3473,2047811,"Ну вот видите! Если бы не Путин, то цены вырос...",1630597064,201281732
3474,2047811,Вообще-то это прерогатива правительства и неко...,1630601474,644528688
3475,2047811,"Путин наш отец родной, об народе думает днём и...",1630607164,577436947


In [13]:
df.to_csv('kommersant_posts.csv', index=False)

In [14]:
data.to_csv('kommersant_comments.csv', index=False)